# 比赛背景

第三届中国AI+创新创业大赛由中国人工智能学会主办，半监督学习目标定位竞赛分赛道要求选手基于少量有标注数据训练模型，使分类网络具有目标定位能力，实现半监督目标定位任务。

中国人工智能学会（Chinese Association for Artificial Intelligence，CAAI）成立于1981年，是经国家民政部正式注册的我国智能科学技术领域唯一的国家级学会，是全国性4A级社会组织，挂靠单位为北京邮电大学；是中国科学技术协会的正式团体会员，具有推荐“两院院士”的资格。

中国人工智能学会目前拥有51个分支机构，包括43个专业委员会和8个工作委员会，覆盖了智能科学与技术领域。学会活动的学术领域是智能科学技术，活动地域是中华人民共和国全境，基本任务是团结全国智能科学技术工作者和积极分子通过学术研究、国内外学术交流、科学普及、学术教育、科技会展、学术出版、人才推荐、学术评价、学术咨询、技术评审与奖励等活动促进我国智能科学技术的发展，为国家的经济发展、社会进步、文明提升、安全保障提供智能化的科学技术服务。

中国“AI+”创新创业大赛由中国人工智能学会发起主办，是为了配合实施创新驱动助力工程，深入开展服务企业技术创新活动，进一步提高我国文化建设和实践创新能力，展示智能科学与技术等相关学科建设的新经验、新成果，促进专业内涵的建设而发起的综合性大赛平台。

飞桨PaddlePaddle作为中国首个自主研发、功能完备、开源开放的产业级深度学习平台，为本次比赛的参赛选手提供了集深度学习核心训练和推理框架、基础模型库、端到端开发套件和丰富的工具组件于一体的一站式服务。百度大脑AI Studio作为官方指定且唯一的竞赛日常训练平台，为参赛选手提供高效的学习和开发环境，更有亿元Tesla V100算力免费赠送，助力选手取得优异成绩。

[比赛链接](https://aistudio.baidu.com/aistudio/competition/detail/78)

# 简要介绍
## 赛题背景
半监督学习(Semi-Supervised Learning)是指通过大量无标记数据和少量有标记数据完成模型训练，解决具有挑战性的模式识别任务。近几年，随着计算硬件性能的提升和大量大规模标注数据集的开源，基于深度卷积神经网络(Deep Convolutional Neural Networks, DCNNs)的监督学习研究取得了革命性进步。然而，监督学习模型的优异性能要以大量标注数据作为支撑，可现实中获得数量可观的标注数据十分耗费人力物力(例如，获取像素级标注数据)。于是， 半监督学习逐渐成为深度学习领域的热门研究方向，只需要少量标记数据就可以完成模型训练过程，更适用于现实场景中的各种任务。
## 比赛任务
本次比赛要求选手基于少量有标签的数据训练模型，使分类网络具有目标定位能力，实现半监督目标定位任务。每- -位参赛选手仅可以使用ImageNet大型视觉识别竞赛(LSVRC)的训练集图像作为训练数据，其中有标签的训练数据仅可以使用大赛组委会提供的像素级标注。
## 数据集介绍
* 训练数据集包括50,000幅像素级标注的图像，共包含500个类，每个类100幅图像;
* A榜测试数据集包括1 1,878幅无标注的图像;
* B榜测试数据集包括10,989幅无标注的图像。
## 评价指标
本次比赛使用loU曲线作为评价指标，即利用预测的目标的定位概率图，计算不同阈值下预测结果与真实目标之间的IoU分数，最后取一个最高点作为最终的分数。在理想状态下，loU曲线最高值接近1.0,对应的阈值为255,因为阈值越高，目标对象与背景的对比度越高。
##  网络介绍
本示例简要介绍如何通过飞桨开源框架，实现图像分割。这里我们是采用了一个在图像分割领域比较熟知的U-Net网络结构，是一个基于FCN做改进后的一个深度学习网络，包含下采样（编码器，特征提取）和上采样（解码器，分辨率还原）两个阶段，因模型结构比较像U型而命名为U-Net。

# 实现
## 解压数据集
默认解压到/data目录下，该目录会在每次进入环境时重置，但可以节省项目启动时间。

In [1]:
!unzip -oq /home/aistudio/data/data95249/train_image.zip -d data/
!unzip -oq /home/aistudio/data/data95249/train_50k_mask.zip -d data/
!unzip -oq /home/aistudio/data/data95249/第一阶段test.zip -d data/

## 导入所需库

In [2]:
import os
import numpy as np
from paddle.io import Dataset,DataLoader
from paddle.vision import transforms as T
from paddle.nn import functional as F
import cv2
import paddle
import matplotlib.pyplot as plt
import paddle.nn as nn
from tqdm import tqdm

## 定义超参

In [3]:
#验证集的数量
eval_num=1000
#所有图像的大小
image_size=(224,224)
#训练图片路径
train_images_path="data/train_image"
#标签路径
label_images_path="data/train_50k_mask"
#测试图片路径
test_images_path="data/val_image"
#批大小
batch_size=16

## 获取所有图片的路径，并分为images和labels两组

In [4]:
#封装好的获取所有图片路径的函数
from utils import get_path
from utils import get_test_data
#扩展一维，以便将images与labels合并
images=np.expand_dims(np.array(get_path(train_images_path)),axis=1)
labels=np.expand_dims(np.array(get_path(label_images_path)),axis=1)
data=np.array(np.concatenate((images,labels),axis=1))
#打乱数据，同时也不会影响images与labels的对应关系
np.random.shuffle(data)
#分割数据集
train_data=data[:-eval_num,:]
eval_data=data[-eval_num:,:]
test_data=get_test_data(test_images_path)
print(train_data.shape)
print(eval_data.shape)
print(test_data.shape)

## 封装数据预处理函数

In [5]:
'''
封装数据预处理函数，其中训练集与验证集、测试集的函数不同，
训练集可以加入图像色彩的调整，但如果要加入水平翻转，缩放等方法，
注意要同时对label也做同样处理
验证集和测试集可以将图像色彩的调整去掉，因为加入数据预处理的原因是
扩大特征空间，使得模型有更好的拟合能力，但验证集就没必要扩大数据的特征空间了
'''
train_transform=T.Compose([
            T.Resize(image_size),
            T.ColorJitter(0.1,0.1,0.1,0.1),
            T.Transpose(),
            T.Normalize(mean=0.,std=255.)
        ])
eval_transform=T.Compose([
            T.Resize(image_size),
            T.Transpose(),
            T.Normalize(mean=0.,std=255.)
        ])

## 定义数据读取器

In [26]:
from utils import show_images
class ImageDataset(Dataset):
    def __init__(self,path,transform):
        super(ImageDataset, self).__init__()
        self.path=path
        self.transform=transform

    def _load_image(self,path):
        '''
        该方法作用为通过路径获取图像
        '''
        img=cv2.imread(path)
        img=cv2.resize(img,image_size)
        return img

    def __getitem__(self,index):
        '''
        这里之所以不对label使用transform，因为观察数据集发现label的图像矩阵主要为0或255
        但偶尔也有0-1的值，所以要对label分情况处理
        而对data都进行transform是因为data都是彩色图片，图像矩阵皆为0-255，所以可以统一处理
        '''
        path=self.path[index]
        if len(path)==2:
            data_path,label_path=path
            data,label=self._load_image(data_path),self._load_image(label_path)
            data,label=self.transform(data),label
            label = label.transpose((2, 0, 1))
            label = label[0, :, :]
            label = np.expand_dims(label, axis=0)
            if True in (label>1):
                label=label/255.
            label = label.astype("int64")
            return data,label
        
        if len(path)==1:
            data=self._load_image(path[0])
            data=self.transform(data)
            return data

    def __len__(self):
        return len(self.path)

#获取数据读取器
train_dataset=ImageDataset(train_data,train_transform)
eval_dataset=ImageDataset(eval_data,eval_transform)
test_dataset=ImageDataset(test_data,eval_transform)
train_dataloader=DataLoader(train_dataset,batch_size=batch_size,shuffle=True,drop_last=True)
eval_dataloader=DataLoader(eval_dataset,batch_size=batch_size,shuffle=True,drop_last=True)
#观察数据读取是否正常
data,label=next(train_dataloader())
show_images([data[0],label[0]])

## 定义网络结构

In [ ]:
from layer import Encoder,Residual
class ImageNet(nn.Layer):
    def __init__(self,num_classes):
        super(ImageNet, self).__init__()
        '''
        encoders用于下采样，decoders用于上采样(实际上为pixelshuffle加下采样)
        '''
        channel=32
        self.encoders=nn.Sequential(
            nn.Conv2D(3,channel*2,5,2,padding="same"),
            nn.BatchNorm(channel*2),
            nn.ReLU(),
            Encoder(channel*2,channel*8),
            Encoder(channel*8,channel*32)
        )

        self.decoders=nn.Sequential(
            nn.PixelShuffle(32),
            Encoder(1,channel*2),
            Encoder(channel*2,channel),
            nn.Conv2D(channel,num_classes,5,padding="same")
        )

    def forward(self,input):
        y=self.encoders(input)
        y=self.decoders(y)
        return y

network=ImageNet(2)
x=paddle.randn([1,3,224,224])
y=network(x)
print(y.shape)

## 使用高层API开启训练

In [ ]:
!mkdir net_params/

In [ ]:
model=paddle.Model(network)
opt=paddle.optimizer.Momentum(learning_rate=1e-4,parameters=model.parameters(),weight_decay=1e-2)
model.prepare(opt, paddle.nn.CrossEntropyLoss(axis=1))
model.fit(train_dataloader, eval_dataloader, epochs=10,verbose=2,save_dir="./net_params",log_freq=200)

## 预测

In [ ]:
!mkdir data/val_label/

In [ ]:
checkpoint_path="./net_params/final"
save_dir="data/val_label"
model=ImageNet(2)
model=paddle.Model(model)
model.load(checkpoint_path)
for i,img in tqdm(enumerate(test_dataset)):
    img=paddle.to_tensor(img).unsqueeze(0)
    predict=np.array(model.predict_batch(img)).squeeze(0).squeeze(0)
    predict=predict.argmax(axis=0)
    image_path=test_dataset.path[i]
    path_lst=image_path[0].split("/")
    save_path=os.path.join(save_dir,path_lst[-1][:-5])+".jpg"
    cv2.imwrite(save_path,predict*255)

# 总结
本基线重在为诸位划水员提供一个参考思路，因此没有进行进一步的优化，分数比较低。换言之，提升空间很大，涨点tips为：
* 更换网络
* 更换激活函数
* 使用visualdl调参
* 集成学习

[最后欢迎大家三连关注一波丫](https://aistudio.baidu.com/aistudio/personalcenter/thirdview/345331)